In [1]:
import gurobipy as gp
from gurobipy import GRB, quicksum
import pandas as pd

In [2]:
df = pd.read_excel('Railway services-2024.xlsx', index_col='Trip')[:200]
df['P_c'] = df['Demand(μ)']
df['L_c'] = 300 
df.loc[df['Line'] == 400, 'L_c'] = 200

display(df)

/var/folders/4g/c51vpg2j0nj65hpt4ffnh7xm0000gn/T/ipykernel_34313/3127984627.py:1: FutureWarning: The behavior of obj[i:j] with a float-dtype index is deprecated. In a future version, this will be treated as positional instead of label-based. For label-based slicing, use obj.loc[i:j] instead
  df = pd.read_excel('Railway services-2024.xlsx', index_col='Trip')[:200]


,Departure Time,Arrival Time,From,To,Demand(μ),Demand(σ),Line,P_c,L_c
Trip,,,,,,,,,
1.0,07:00:00,07:46:00,M,A,327.0,64.0,100.0,327.0,300
2.0,07:00:00,07:57:00,H,M,936.0,308.0,800.0,936.0,300
3.0,07:02:00,07:22:00,M,B,461.0,252.0,200.0,461.0,300
4.0,07:04:00,08:03:00,M,J,428.0,10.0,1000.0,428.0,300
5.0,07:06:00,08:13:00,M,F,449.0,124.0,600.0,449.0,300
...,...,...,...,...,...,...,...,...,...
196.0,08:57:00,09:30:00,D,M,384.0,314.0,400.0,384.0,200
197.0,08:58:00,09:26:00,M,D,395.0,366.0,400.0,395.0,200
198.0,08:58:00,09:33:00,M,E,331.0,436.0,500.0,331.0,300


In [3]:
# Sets for model
rolling_stock_types = ['OC', 'OH']
train_services = df.index.to_list()

# Initialise model
model = gp.Model("Train_Scheduling")

# Constants
costs = {'OC': 260000, 'OH': 210000}
capacity = {'OC': 620, 'OH': 420}
length = {'OC': 100, 'OH': 70}

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-07


In [9]:
# Decision Variables
N = model.addVars(rolling_stock_types, train_services, vtype=GRB.INTEGER, name='N')

# Objective Function
model.setObjective(quicksum(costs[u] * N[u, c] for u in rolling_stock_types for c in train_services), GRB.MINIMIZE)

# Constraints
# Passenger Demand
demand_constraints = model.addConstrs((quicksum(capacity[u] * N[u, c] for u in rolling_stock_types) >= df.loc[c, 'P_c'] for c in train_services), name="Demand")

# Train Length
length_constraints = model.addConstrs((quicksum(length[u] * N[u, c] for u in rolling_stock_types) <= df.loc[c, 'L_c'] for c in train_services), name="Length")

# Proportionality - ensuring the number of OC and OH units are within 25% of each other
total_OC = quicksum(N['OC', c] for c in train_services)
total_OH = quicksum(N['OH', c] for c in train_services)
model.addConstr(total_OC <= 1.25 * total_OH, "OC_to_OH_proportion")
model.addConstr(total_OH <= 1.25 * total_OC, "OH_to_OC_proportion")

# Solve the model
model.optimize()

# Output the solution
if model.Status == GRB.OPTIMAL:
    print("Optimal solution found:")
    results = {(u, c): N[u, c].X for u in rolling_stock_types for c in train_services}

    # Convert to DataFrame
    unit_df = pd.DataFrame.from_dict(results, orient='index', columns=['Units'])
    unit_df.index = pd.MultiIndex.from_tuples(unit_df.index, names=('Rolling Stock Type', 'Service'))
    unit_df = unit_df.reset_index()
    unit_df = unit_df.pivot(index='Service', columns='Rolling Stock Type', values='Units')

    display(unit_df)
    print(f"Minimized Total Cost: {model.objVal}")
elif model.Status in [GRB.INF_OR_UNBD, GRB.INFEASIBLE, GRB.UNBOUNDED]:
    print("Model cannot be solved to optimality due to infeasibility or unboundedness.")
else:
    print(f"Optimization was stopped with status {model.Status}")

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.1.0 23B2073)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 1206 rows, 1200 columns and 4800 nonzeros
Model fingerprint: 0x993c40c5
Variable types: 0 continuous, 1200 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [2e+05, 3e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+03]

MIP start from previous solve produced solution with objective 8.757e+07 (0.01s)
MIP start from previous solve produced solution with objective 7.341e+07 (0.01s)
MIP start from previous solve produced solution with objective 7.315e+07 (0.01s)
MIP start from previous solve produced solution with objective 7.294e+07 (0.01s)
MIP start from previous solve produced solution with objective 7.299e+07 (0.02s)
MIP start from previous solve produced solution with objective 7.309e+07 (0.02s)
Loaded MIP sta

Rolling Stock Type,OC,OH
Service,,
1.0,0.0,1.0
2.0,1.0,1.0
3.0,1.0,0.0
4.0,1.0,0.0
5.0,1.0,0.0
...,...,...
196.0,0.0,1.0
197.0,0.0,1.0
198.0,0.0,1.0


Minimized Total Cost: 72940000.0
